In [1]:
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"

In [2]:
import os
import pandas as pd
from pathlib import Path
from urllib.error import HTTPError

In [3]:
md = pd.read_csv('metadata.csv')

# Parse, Save, Load the Data

In [4]:
src = [name.replace('/', '-') for name in md['cell_id'].to_list() if 'SNL' in name]
repos = (r'https://www.batteryarchive.org/data/' + url + r'_timeseries.csv' for url in src)


def load_battery_data(path):
    """Downloads the battery data, encoded as a pickle object (*.pkl)
    into the datasets directory
    
    Data source -- @www.batteryarchive.org
    """
    
    try:
        os.chdir(path / r'483_ML_BatteryLife/')
        data_path = r'datasets/battery.pkl'
        if not Path('datasets/').is_dir():
            Path("datasets").mkdir(parents=True, exist_ok=True)
        if not Path(data_path).is_file():
            df = pd.concat([pd.read_csv(url) for url in repos])
            df.to_pickle(data_path)
            print("Download Complete!")
        return pd.read_pickle(data_path)
    except HTTPError:
        print("Download Failed!")

In [5]:
battery = load_battery_data(Path(r'/home/jason/'))  # change path to Github repository

# Look at the Data

In [6]:
battery.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23888882 entries, 0 to 80104
Data columns (total 11 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0   Date_Time                    object 
 1   Test_Time (s)                float64
 2   Cycle_Index                  float64
 3   Current (A)                  float64
 4   Voltage (V)                  float64
 5   Charge_Capacity (Ah)         float64
 6   Discharge_Capacity (Ah)      float64
 7   Charge_Energy (Wh)           float64
 8   Discharge_Energy (Wh)        float64
 9   Environment_Temperature (C)  float64
 10  Cell_Temperature (C)         float64
dtypes: float64(10), object(1)
memory usage: 2.1+ GB


In [7]:
battery.head()

,Date_Time,Test_Time (s),Cycle_Index,Current (A),Voltage (V),Charge_Capacity (Ah),Discharge_Capacity (Ah),Charge_Energy (Wh),Discharge_Energy (Wh),Environment_Temperature (C),Cell_Temperature (C)
0,2018-02-01 10:02:10.002962,10.002,1.0,0.0,2.940,0.0,0.0,0.0,0.0,15.163,15.375
1,2018-02-01 10:02:20.016433,20.016,1.0,0.0,2.941,0.0,0.0,0.0,0.0,15.148,15.437
2,2018-02-01 10:02:30.031429,30.031,1.0,0.0,2.941,0.0,0.0,0.0,0.0,15.087,15.375
3,2018-02-01 10:02:40.046426,40.046,1.0,0.0,2.941,0.0,0.0,0.0,0.0,15.240,15.375
4,2018-02-01 10:02:50.061462,50.061,1.0,0.0,2.941,0.0,0.0,0.0,0.0,15.087,15.375


In [8]:
battery.describe()

,Test_Time (s),Cycle_Index,Current (A),Voltage (V),Charge_Capacity (Ah),Discharge_Capacity (Ah),Charge_Energy (Wh),Discharge_Energy (Wh),Environment_Temperature (C),Cell_Temperature (C)
count,2.388888e+07,2.388888e+07,2.388888e+07,2.388888e+07,2.388888e+07,2.388888e+07,2.388888e+07,2.388888e+07,2.336520e+07,2.336520e+07
mean,1.613473e+07,2.841530e+03,-1.981644e-01,3.446082e+00,9.805721e-01,3.515892e-01,3.558338e+00,1.218359e+00,2.491242e+01,2.591522e+01
std,1.366815e+07,3.178384e+03,1.398625e+00,3.230248e-01,1.092508e+00,7.203539e-01,3.933567e+00,2.461379e+00,4.940035e+00,4.842974e+00
min,2.100000e-02,1.000000e+00,-9.998000e+00,-4.646000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.234700e+01,5.605000e+00
25%,4.554111e+06,5.210000e+02,-1.499000e+00,3.280000e+00,4.500000e-01,0.000000e+00,1.525000e+00,0.000000e+00,2.443900e+01,2.447900e+01
50%,1.292059e+07,1.853000e+03,1.000000e-03,3.386000e+00,6.410000e-01,0.000000e+00,2.260000e+00,0.000000e+00,2.475900e+01,2.503300e+01
75%,2.519087e+07,3.684000e+03,5.490000e-01,3.614000e+00,1.036000e+00,4.400000e-01,3.503000e+00,1.513000e+00,2.505700e+01,2.637400e+01
max,6.618272e+07,1.917500e+04,9.999000e+00,9.983000e+00,1.927600e+01,9.735000e+00,6.382100e+01,3.149800e+01,3.729300e+01,5.536100e+01


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

plt.rc('font', size=14)
plt.rc('axes', labelsize=14, titlesize=10)
plt.rc('legend', fontsize=14)
plt.rc('xtick', labelsize=8)
plt.rc('ytick', labelsize=8)

battery.hist(bins=50, figsize=(10, 10))
plt.show()